In [2]:
# Documentation
# STAT3612 Statistical Machine Learning, SEM2, 2022
# Group Project
# Name: Lau Yan Chun Chris
# UID:3035790941

# This python notebook is mainly for the part of the feature selection of the group project

In [1]:
! pip install numpy pandas matplotlib seaborn sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
## Part (A) Feature Selection

In [2]:
#read X_train first 3 rows for header
df_X_header = pd.read_csv("X_train.csv", dtype = str, nrows = 3,header = None)

#combine df_X_header by row
X_header = []
for ic in range(len(df_X_header.columns)):
    temp = ""
    for ir in range(len(df_X_header)):
        if(ir != 0):
            temp += "&"
        temp += df_X_header.iloc[ir,ic]
    X_header.append(temp)
    


#read X_train csv files except first 3 rows 
X_train = pd.read_csv("X_train.csv",skiprows = 3,header = None)

X_valid = pd.read_csv("X_valid.csv",skiprows = 3,header = None)

X_test = pd.read_csv("X_test.csv",skiprows = 4,header = None)

#combine header and the data together
X_train.columns = X_header
X_valid.columns = X_header
X_test.columns = X_header

#read Y_train csv file
Y_train = pd.read_csv("Y_train.csv")
Y_valid = pd.read_csv("Y_valid.csv")

In [3]:
train_id = X_train.iloc[:,0]
valid_id = X_valid.iloc[:,0]
test_id  = X_test.iloc[:,0]
X_train = X_train.iloc[:,1:]
X_valid = X_valid.iloc[:,1:]
X_test  = X_test.iloc[:,1:]
X_header = X_header[1:]
print("X_train:",X_train.shape)
print("X_valid:",X_valid.shape)
print("X_test:",X_test.shape)
print("Y_train:",Y_train.shape)
print("Y_valid:",Y_valid.shape)

X_train: (16760, 7488)
X_valid: (2394, 7488)
X_test: (4790, 7488)
Y_train: (16760, 2)
Y_valid: (2394, 2)


In [ ]:
#Three principle to conduct the features Selection
    #1. Only select the features mean of the variable by time
    #2. Only select the either one feature among the features group with low covariance (< 0.9)
    #3  Only select the features with non-zero weights under Simple Logistic Regression Model with L1 Regularization
    

In [4]:
def featureSelectByMean(df_feat,df_header,cols = None):
    if (cols == None):
        cols = np.full((df_feat.shape[1],), False, dtype = bool)
    for ic in range(len(df_header.columns)):
        if (df_header.iloc[1,ic] == "mean"):
            cols[ic] = True
    
    print("Select features with mean value:")
    print("Original:",df_feat.shape[1],"\tSelected:",np.sum(cols))        
    return df_feat.iloc[:,cols]
    

X_train = featureSelectByMean(X_train,df_X_header)
X_train.describe()

Select features with mean value:
Original: 7488 	Selected: 2496


,alanine aminotransferase&mean&1,alanine aminotransferase&mean&2,alanine aminotransferase&mean&3,alanine aminotransferase&mean&4,alanine aminotransferase&mean&5,alanine aminotransferase&mean&6,alanine aminotransferase&mean&7,alanine aminotransferase&mean&8,alanine aminotransferase&mean&9,alanine aminotransferase&mean&10,...,white blood cell count urine&mean&15,white blood cell count urine&mean&16,white blood cell count urine&mean&17,white blood cell count urine&mean&18,white blood cell count urine&mean&19,white blood cell count urine&mean&20,white blood cell count urine&mean&21,white blood cell count urine&mean&22,white blood cell count urine&mean&23,white blood cell count urine&time_since_measured&0
count,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,...,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000,16760.000000
mean,-0.065219,-0.064588,-0.064803,-0.064101,-0.064031,-0.063712,-0.063905,-0.064175,-0.064640,-0.064856,...,-0.000300,-0.000252,-0.000329,-0.000262,-0.000249,-0.000257,-0.000282,-0.000300,-0.000448,269.869093
std,0.423142,0.428927,0.427698,0.435464,0.436145,0.438468,0.435569,0.432840,0.426678,0.420549,...,0.339446,0.338852,0.338696,0.338709,0.338702,0.338702,0.338711,0.338711,0.338265,315.670695
min,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,-0.279057,...,-0.316046,-0.316046,-0.316046,-0.316046,-0.316046,-0.316046,-0.316046,-0.316046,-0.316046,0.000000
25%,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,-0.248043,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,154.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,360.000000
max,10.413320,10.413320,10.413320,10.413320,10.413320,10.413320,10.413320,10.413320,10.413320,10.413320,...,11.783246,11.783246,11.783246,11.783246,11.783246,11.783246,11.783246,11.783246,11.783246,2232.000000


In [ ]:
def featureSelectByCorr(df_feat,threshold = 0.9):
    corr = df_feat.corr()
    
    cols = np.full((corr.shape[0],), True, dtype = bool)
    for i in range(corr.shape[0],0,-1):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= threshold:
                if(cols[j]):
                    cols[j] = False
    print("Select features by filtering features with high covariance","(>= "+str(threshold)+")",":")
    print("Original:",df_feat.shape[1],"\tSelected:",np.sum(cols))
    
    return df_feat.iloc[:,cols]

X_train = featureSelectByCorr(X_train,0.9)
#for the part of data visualization
X_train_init = X_train.copy()

X_train.describe()

In [ ]:
#visualizing the features to check its distribution to determine whether normalization or standardization is more suitable for feature scaling

check_ID = 365

sns.displot(X_train.iloc[:,check_ID], color = 'g', kde  = True).set(title = X_train.columns[check_ID])

plt.show()

In [ ]:
#Standardlization of features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = pd.DataFrame(scaler.fit_transform(X_train))
X_train_std.columns = X_train.columns
X_train_std.describe()

In [ ]:
#Feature selection by simple logistic regression with L1 regulation
from sklearn.linear_model import LogisticRegression

def CheckSelectBySLR_L1(X_std,Y):
    C = [1e-3,2e-3,3e-3,4e-3,5e-3,6e-3,7e-3,8e-3,9e-3,1e-2,1.1e-2,1.2e-2,1.3e-2,1.4e-2]
    print("Features selected after applying SLR model with L1 regularization:")
    n_feat = []
    for n_c in C:
        clf = LogisticRegression(penalty = 'l1',C = n_c, solver = 'liblinear')
        clf.fit(X_std,Y)
    
        cols_l1 = []
        for i in range(clf.coef_.shape[1]):
            if clf.coef_[0,i] != 0.0:
                cols_l1.append(True)
            else:
                cols_l1.append(False)
        n_feat.append(np.sum(cols_l1))
        print("L1 C value:",n_c, "\tNumber of features:",clf.coef_.shape[1], "\tSelected:",np.sum(cols_l1))

    nc_log = list(map(np.log10,C))
    plt.xlabel("L1 Regularization parameter C")
    plt.ylabel("Number of features")
    plt.title("Number of features remain versus Lasso regression parameter")
    plt.plot(nc_log, n_feat, marker = "o",color = "g")

CheckSelectBySLR_L1(X_train_std,Y_train.iloc[:,1])

In [ ]:
def featureSelectBySLR_L1(X,Y,C = 0.01):
    clf = LogisticRegression(penalty = 'l1',C = C, solver = 'liblinear')
    clf.fit(X,Y)
    cols_l1 = []
    for i in range(clf.coef_.shape[1]):
        if clf.coef_[0,i] != 0.0:
            cols_l1.append(True)
        else:
            cols_l1.append(False)
    return X.iloc[:,cols_l1]

X_train_select = featureSelectBySLR_L1(X_train_std,Y_train.iloc[:,1], C = 0.005)
print(X_train_select.shape)
print(X_train_select.columns)

In [ ]:
def featureSelect(header, df_select,df_X):
    cols = np.full(len(header), False, dtype = bool)
    for i,feat in enumerate(header):
        if feat in df_select.columns:
            cols[i] = True
    return df_X.iloc[:,cols]

X_valid_select = featureSelect(X_header, X_train_select,X_valid)
X_valid_select.columns = X_train_select.columns
print(X_valid.shape)
print(X_valid_select.shape)

In [ ]:
#1 Task 1
#Use simple Logistic Regression with L2 regulation as model

def viscualizeSLR_arr(X_train,Y_train,X_test,Y_test,
                      l1_label = str(1e-3),C_l2 = None,color = 'b'):
    
    if(C_l2 == None):
        C_l2 = [1e-6,1e-3,1,1e3,1e6,1e9]
    accuracy = []
    
    for ci_l2 in C_l2:
        clf = LogisticRegression(penalty = 'l2', C = ci_l2, solver = 'liblinear')
        clf.fit(X_train,Y_train)
        accuracy.append(clf.score(X_test,Y_test))

    nc_log = list(map(np.log10,C_l2))
    plt.plot(nc_log, accuracy, marker = "o",color = color,label = l1_label)
    print("Maximized accuracy :",np.max(accuracy),"at C_L2:",C_l2[accuracy.index(np.max(accuracy))])

viscualizeSLR_arr(X_train_select, Y_train.iloc[:,1], X_valid_select, Y_valid.iloc[:,1])

In [ ]:
C_l1 = [2e-3,3e-3,4e-3,5e-3,6e-3,7e-3]
C_l2 = [1e-6,1e-4,1e-2,1,1e2]

def ViscualizeSLR_compare(C_l1, C_l2):
    plot_color = ['r','g','b','c','m','y','k']
    plot_color = plot_color[:len(C_l1)]
    
    for ci_l1, color in zip(C_l1,plot_color):
        print("using L1 regularization with C value:",ci_l1)
        X_train_selected = featureSelectBySLR_L1(X_train_std,Y_train.iloc[:,1], C = ci_l1)
        X_valid_selected = featureSelect(X_header, X_train_selected,X_valid)
        X_valid_selected.columns = X_train_selected.columns
        
        viscualizeSLR_arr(X_train_selected, Y_train.iloc[:,1], X_valid_selected, Y_valid.iloc[:,1],
                          l1_label = ("L1:" + str(ci_l1)),C_l2 = C_l2,color = color)
    
    plt.ylabel("Accuracy")
    plt.xlabel("log10.(C_L2)")
    plt.legend()
    plt.title("Best accuracy versus C_L2 with different C_L1")
    
ViscualizeSLR_compare(C_l1,C_l2)

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

clf1 = LogisticRegression()
clf2 = DecisionTreeClassifier(random_state = 42)
clf3 = KNeighborsClassifier()
clf4 = RandomForestClassifier(random_state = 42)
clf5 = GradientBoostingClassifier(random_state = 42)

param1 = {'classifier__C': [1e-6,1e-4,1e-2,1,1e2],
          'classifier__penalty': ['l2'],
          'classifier' : [clf1]
         }

param2 = {'classifier__max_depth': [5,10,25,None],
          'classifier__min_samples_split': [2,5,10],
          'classifier__class_weight': [None,{0:1,1:5},{0:1,1:10}],
          'classifier' : [clf2]
         }

param3 = {'classifier__n_neighbors': [2,5,10,25,50],
         'classifier' : [clf3]
         }

param4 = {'classifier__n_estimators': [10,50,100,250],
         'classifier__max_depth': [5,10,20],
         'classifier__class_weight': [None,{0:1,1:5},{0:1,1:10}],
         'classifier' : [clf4]
         }

param5 = {'classifier__n_estimators': [10,50,100,250],
         'classifier__max_depth': [5,10,20],
         'classifier' : [clf5]
         }

pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3, param4, param5]

X_train_select = featureSelectBySLR_L1(X_train_std,Y_train.iloc[:,1], C = 5e-3)

gs = GridSearchCV(pipeline, params, cv = 5, n_jobs = -1, scoring = 'roc_auc', verbose = 2).fit(X_train_select, Y_train.iloc[:,1])



Fitting 5 folds for each of 94 candidates, totalling 470 fits


In [49]:
gs.best_params_

{'classifier': RandomForestClassifier(class_weight={0: 1, 1: 5}, max_depth=20,
                        n_estimators=250, random_state=42),
 'classifier__class_weight': {0: 1, 1: 5},
 'classifier__max_depth': 20,
 'classifier__n_estimators': 250}

In [ ]:
gs.best_score_

In [ ]:
Y_valid_hat = gs.predict(X_valid)

In [ ]:
Y_test_hat = gs.predict(X_test)
res_t1 = pd.DataFrame({"id": test_id,"Prediction" : Y_test_hat})
res_t1.to_csv('Submission_TaskOne.csv')

In [ ]:
X_train_init_select = featureSelect(X_header, X_train_select,X_train_init)
X_valid_select= featureSelect(X_header, X_train_select,X_valid)
X_test_select = featureSelect(X_header, X_train_select,X_test)


In [ ]:
from sklearn.manifold import TSNE
def visualizeResult(X_train,Y_train,
                    X_test, y_test_hat):
    
    tSNE = TSNE(n_components = 2, learning_rate = 'auto',init = 'random')
    xt_train = TSNE.fit_transform(X_train)
    xt_test = TSNE.transform(xt_test)
    
    labels = ["train","test"]
    data_set = [(xt_train,Y_train),
                (xt_test,y_test_hat)]
    
    plt.figure()
    for label,(x,y) in zip(labels,data_set):
        plt.scatter(x[y == 0,0], x[y== 0,1],s = 5, label = label+"0")
        plt.scatter(x[y == 1,0], x[y== 1,1],s = 5, label = label+"1",alpha=0.5)
    plt.legend()
    plt.show()

visualizeResult(x_train_init_select,Y_train.iloc[:,1],x_test_select, Y_valid_hat)

[CV] END classifier=LogisticRegression(), classifier__C=1e-06, classifier__penalty=l2; total time=   0.1s
[CV] END classifier=LogisticRegression(), classifier__C=0.0001, classifier__penalty=l2; total time=   0.1s
[CV] END classifier=LogisticRegression(), classifier__C=0.0001, classifier__penalty=l2; total time=   0.0s
[CV] END classifier=LogisticRegression(), classifier__C=0.01, classifier__penalty=l2; total time=   0.1s
[CV] END classifier=LogisticRegression(), classifier__C=1, classifier__penalty=l2; total time=   0.1s
[CV] END classifier=LogisticRegression(), classifier__C=100.0, classifier__penalty=l2; total time=   0.1s
[CV] END classifier=DecisionTreeClassifier(random_state=42), classifier__class_weight=None, classifier__max_depth=5, classifier__min_samples_split=2; total time=   0.1s
[CV] END classifier=DecisionTreeClassifier(random_state=42), classifier__class_weight=None, classifier__max_depth=5, classifier__min_samples_split=2; total time=   0.1s
[CV] END classifier=DecisionT